In [1]:
from rodan_pretrained import RodanPretrained
model = RodanPretrained(pretrained_lr=1e-4, my_layers_lr=1e-3, warmup_steps = 10000)

In [2]:
from dataloading2 import nanopore_datamodule
from data_utils.split_methods import get_kfold_split_func, get_default_split
dm = nanopore_datamodule(
    pos_files='pos_2022', 
    neg_files='neg_2022', 
    verbose=1, 
    batch_size=64, 
    valid_limit=10000, 
    workers=32,
    split_method=get_kfold_split_func(total_k=5, current_k=3),
    window=4096,
)

valid files indicies
[8, 14, 17, 18, 25, 41, 43, 49, 53, 54, 56, 58, 59, 62, 68, 69, 74, 80, 83, 90, 92, 93, 94, 102, 116, 118, 136, 142, 145, 147, 155, 158, 160, 162, 163, 165, 169, 171, 174, 175, 176, 179, 180, 193, 197, 198, 200, 203, 209, 212, 228, 236, 243, 251, 265]
[12, 18, 19, 21, 22, 26, 38, 47, 52, 53, 54, 55, 57, 59, 65, 66, 68, 75, 76, 81, 82, 83, 90, 91, 102, 107, 113, 116, 121, 125, 127, 128, 134, 136, 139, 141, 144, 145, 154, 156, 157, 175, 178, 179, 181, 183, 186, 189, 195, 208, 210, 211, 220, 224, 226, 235, 240, 250, 258, 270, 278, 279, 282, 283, 286, 287, 309, 313, 324, 326, 334, 339, 341, 351, 353, 360, 361, 362, 368, 380, 385, 389, 391, 394, 397, 399, 411, 425, 430, 433, 440, 445, 449, 450, 468, 487, 489, 496, 504, 508, 511, 512, 514, 515]
train files indicies
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 15, 16, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 44, 45, 46, 47, 48, 50, 51, 52, 55, 57, 60, 61, 63, 64, 65, 66, 67, 7

In [3]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import ModelCheckpoint

experiment_name = 'rodan_2022_kfold_3of5'
checkpoint_callback = ModelCheckpoint(dirpath=f"checkpoints_pl/{experiment_name}", save_top_k=2, monitor="valid_loss")
logger = CometLogger(api_key="TEVQbgxxvilM1WdTyqZLJ57ac", project_name='RNAModif', experiment_name=experiment_name) 
trainer= pl.Trainer(
    max_steps = 1000000, logger=logger, accelerator='gpu',
    auto_lr_find=False, val_check_interval=500,  
    log_every_n_steps=500, benchmark=True, precision=16,
    callbacks=[checkpoint_callback])#, profiler="simple")

trainer.fit(model, dm)


CometLogger will be initialized in online mode
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /home/jovyan/RNAModif/checkpoints_pl/rodan_2022_kfold_3of5 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | Sequential      | 10.9 M
1 | acc   | Accuracy        | 0     
2 | cm    | ConfusionMatrix | 0     
------------------------------------------
10.9 M    Trainable params
0         Non-trainable params
10.9 M    Total params
21.760    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: 2 nan values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experime

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:657: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
